In [1]:
#Import necessary packages igraph, scvelo, scanpy, anndata, pandas, os, numpy
!pip install -U scvelo
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%matplotlib inline

import scvelo as scv
scv.logging.print_version()

#test whether igraph is imported
import igraph as ig

import loompy
import numpy as np
import scanpy as sc
import anndata
import pandas as pd
import os
import louvain
cwd = os.getcwd()
cwd

Requirement already up-to-date: scvelo in /usr/local/lib/python3.7/site-packages (0.2.2)


Running scvelo 0.2.2 (python 3.7.7) on 2020-09-08 13:43.


'/Users/whippoorwill/Desktop/Jupyter'

In [2]:


os.listdir(cwd)

['Iterative Clustering with PanoView.ipynb',
 'splicegenes.csv',
 '.DS_Store',
 'ScVelo Split by sample_description.ipynb',
 'ScVelo-mgAVM02-Copy1.ipynb',
 'velocity_Nick',
 'cache',
 'CellPhoneDB.ipynb',
 'out',
 'BMI219_CDT_GroupProject_FinalCode.ipynb',
 'LD1',
 'ScVelo-mgAVM02.ipynb',
 'mgAVM02',
 'ScVelo2-Copy1.ipynb',
 'figures',
 'spliced_unspliced_ratio.csv',
 '.ipynb_checkpoints',
 'ScVelo Split by condition.ipynb',
 'cellphonedb',
 'Velocyto_phi.ipynb',
 'Phi',
 'BMI219',
 'ScVelo2.ipynb']

Combine 4 loom files into 1


In [377]:
#files=os.listdir(cwd)
#newfile='nick.loom'
#loompy.combine(files,newfile,key="Accession")

Import the data file from inside the Jupyter folder as "adata"

In [26]:
adata = scv.read(filename="mgAVM02/mgAVM02.loom",cache=True)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
... storing 'Chromosome' as categorical
... storing 'Strand' as categorical


In [3]:
adata = scv.read("mgAVM02/80neighborsadata.h5ad",cache = True)

If you get a message that "variable names are not unique", run the first line (make_unique). Also set verbosity and figure params for good visualization. 

In [27]:
adata.var_names_make_unique()
scv.settings.verbosity = 4  # show errors(0), warnings(1), info(2) and hints(3)
scv.settings.set_figure_params('scvelo')  # for beautified visualization

In [28]:
# show proportions of spliced/unspliced abundances
scv.utils.show_proportions(adata)
adata

Abundance of ['spliced', 'unspliced']: [0.53 0.47]


AnnData object with n_obs × n_vars = 35259 × 31053 
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'matrix', 'ambiguous', 'spliced', 'unspliced'

Open a file containing the names of the subset of cells that you want to use, if you need the cells subsetted. I will only use the mg subset (from R). 

In [35]:
anno = pd.read_table("mgAVM02/meta.csv",header=0, sep=",")


print(anno['finalclusters'][1])
print(type(anno['finalclusters'][1]))

#make clusters into strings
clusterstring = []
for cluster in anno['finalclusters']:
    cluster = str(cluster)
    clusterstring.append(cluster)

anno['finalclusters'] = clusterstring

#make individual into strings
clusterstring = []
for cluster in anno['sample_description']:
    cluster = str(cluster)
    clusterstring.append(cluster)

anno['sample_description'] = clusterstring
    
print(type(anno['finalclusters'][1]))
print(anno['finalclusters'][1])
l = anno['Cellname']
print(l[1])

print(adata.var_names[1])
print (adata.obs_names[1])

6
<class 'numpy.int64'>
<class 'str'>
6
AAACCCAGTGCCTTTC-LD5RC
Gm37381
LD5LD:AAAGGTAAGGTTCTTGx


In [36]:
len(l)

12330

In [37]:
umap = pd.read_csv("mgAVM02/umap.csv")
pca = pd.read_csv("mgAVM02/pcs_noneuronal.csv")
print(umap[0:2][1:5])
print(pca[0:2][1:5])

                 Cellname    UMAP_1    UMAP_2
1  AAACCCAGTGCCTTTC-LD5RC -2.765974 -4.103992
                 Cellname      PC_1      PC_2      PC_3      PC_4      PC_5  \
1  AAACCCAGTGCCTTTC-LD5RC -3.173488 -8.076785 -2.731364  6.219031 -0.910135   

      PC_6      PC_7     PC_8      PC_9    ...        PC_41     PC_42  \
1 -0.52072  2.977634 -3.14196 -0.343547    ...     0.500033 -1.098383   

      PC_43    PC_44     PC_45   PC_46     PC_47     PC_48     PC_49     PC_50  
1  1.815273 -0.02464 -0.520742 -0.4453  0.490274  1.123172  0.531638  0.896684  

[1 rows x 51 columns]


Each line is formatted differently from the cellnames in the loom file, which is: LD5LD:---------x. The metadata is ---------- - LD5RC. Additionally, each line is imported with a trailing 'n'.

In [443]:
#n = [i.replace('\n','') for i in l] #initialize a new object 'n' that contains each line of 'l' without the trailing \n
#n[0] #read the second line to verify that this worked

In [38]:
l = list(anno['Cellname'])
print(type(l))
l[1]
a = list(umap['Cellname'])
a == l #Check that the cells in the umap list are in the same order as the annotation

<class 'list'>


True

In [39]:
#Now split each line and move the elements around to format the metadata in the same way as the loom object
j=0
for i in l:
    x = i.split("-")
    #a = "NSMG20_" + x[1]+":"+x[0]+"x"
    a = x[1]+":"+x[0]+"x"
    l[j] = a
    j=j+1

In [40]:
anno.index = l
umap.index = l
pca.index = l
#sample_one_index = pd.DataFrame(sample_one.obs.index)
#sample_one = sample_one_index.rename(columns = {0:'Cell ID'}
#umap_ordered = sample_one_index.merge(umap,on="Cell ID")
#umap_ordered = umap_ordered.iloc[:,1:]
#sample_one.obsm['X_umap'] = umap_ordered.values

In [41]:
print(adata.obs_names)
print(l)

Index(['LD5LD:AAAGGTACAGATTTCGx', 'LD5LD:AAAGGTAAGGTTCTTGx',
       'LD5LD:AAAGGGCGTTCGCGTGx', 'LD5LD:AAACGCTTCCACTTTAx',
       'LD5LD:AAAGGTATCAGTAGGGx', 'LD5LD:AAATGGATCAATCTCTx',
       'LD5LD:AAAGGTAAGTCCTACAx', 'LD5LD:AAATGGAGTCCCACGAx',
       'LD5LD:AACAAAGCAGAGATGCx', 'LD5LD:AAACCCAGTCGTACATx',
       ...
       'LD7RC:TTTGATCAGTCCGCCAx', 'LD7RC:TTTGACTCATAACAGAx',
       'LD7RC:TTTGGTTTCTGCGGGTx', 'LD7RC:TTTGTTGCAAATCAGAx',
       'LD7RC:TTTCGATTCCTAGCGGx', 'LD7RC:TTTGTTGTCGATCCCTx',
       'LD7RC:TTTGGAGCACCTTCGTx', 'LD7RC:TTTGACTGTATCGTTGx',
       'LD7RC:TTTCGATTCCAGCAATx', 'LD7RC:TTTGTTGGTGTACATCx'],
      dtype='object', length=35259)
['LD5RC:AAACCCAAGCAACAATx', 'LD5RC:AAACCCAGTGCCTTTCx', 'LD5RC:AAACGAAAGGTGGGTTx', 'LD5RC:AAACGAACAACAAGATx', 'LD5RC:AAACGAAGTAGCTGTTx', 'LD5RC:AAACGAAGTCCAGCCAx', 'LD5RC:AAACGAAGTCTGTGTAx', 'LD5RC:AAACGAATCCTGTAGAx', 'LD5RC:AAACGAATCGTAACACx', 'LD5RC:AAACGCTAGACGGTTGx', 'LD5RC:AAACGCTAGCGTCAGAx', 'LD5RC:AAAGAACGTCGTTGGCx', 'LD5RC:AAAGAACGTT

In [42]:
genes = pd.read_table("mgAVM02/genes_noneuronal.csv",header=0, sep=",")
genes = list(genes['Genes'])
print(adata.var_names)

Index(['Xkr4', 'Gm37381', 'Rp1', 'Sox17', 'Gm37323', 'Mrpl15', 'Rgs20',
       'Npbwr1', '4732440D04Rik', 'Gm26901',
       ...
       'Gm28406', 'Gm29436', 'Gm28407', 'Gm29393', 'Gm21294', 'Gm28672',
       'Gm28670', 'Gm29504', 'Gm20837', 'Gm47283'],
      dtype='object', length=31053)


Now subset the adata object including on the cells that are in the metadata file (n). 

In [43]:
len(genes)
i = 0
k = []
for gene in genes:
    if gene not in adata.var_names:
        k.append(i)
    i = i+1
#genes = genes[adata.var_names]

In [44]:
print(i)
print(k)
genes[k[1]]


5288
[3154, 4917]


'St6galnac2.1'

In [45]:
newgenes = [j for i, j in enumerate(genes) if i not in k]

In [46]:
print(len(genes),len(newgenes))

5288 5286


In [53]:
adata_subset = adata[l,newgenes]


In [55]:
adata = adata_subset
adata
#check whether the number of cells and genes is correct now

View of AnnData object with n_obs × n_vars = 12330 × 5286 
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'matrix', 'ambiguous', 'spliced', 'unspliced'

Now write a file (cellnames.txt) with all the cellnames. This is strictly unnecessary but can be used to check against Seurat cellnames. 

In [56]:
print(adata.var_names[1])
print (adata.obs_names[1])
#with open('cellnames.txt','w') as f:
    #for item in adata.obs_names:
        #f.write("%s\n"%item)

#Check that the lists are ordered correctly
anno.index.all() == adata.obs_names.all()

Pf4
LD5RC:AAACCCAGTGCCTTTCx


True

In [57]:
print(type(anno))
anno[['finalcluster','individual','Cellname']][1:5]

<class 'pandas.core.frame.DataFrame'>


KeyError: "['finalcluster' 'individual'] not in index"

In [457]:
print (adata.obs_keys) #find out whether there is any annotation yet in adata

<bound method AnnData.obs_keys of View of AnnData object with n_obs × n_vars = 6666 × 5860 
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'>


Add in metadata from the text file; needs to be separated by commas and you can use the headings in that file to pull out a column

In [89]:
adata.obs[['nCount_RNA','percent.mito','Cluster','Individual','Condition']] = anno[['nCount_RNA','percent.mito','finalclusters','sample_description','condition']]

In [60]:
#Remove the cellnames column from umap and pca
umap = umap[['UMAP_1','UMAP_2']].values
pca = pca.iloc[:,1:30].values
print(pca[0:2][1:5])

print(type(umap))
print(type(pca))
print(umap)
print(pca)



adata.obsm['X_umap'] = umap
adata.obsm['X_pca'] = pca
adata #check whether attributes have been added

[[-3.1734877  -8.07678463 -2.73136418  6.21903122 -0.91013488 -0.52072022
   2.97763436 -3.14196017 -0.34354731  2.68262892  3.26429161 -4.22862849
  -4.06394436  2.72476348 -1.10939878 -1.34068098  0.70610778  0.06905808
   1.11374358 -0.8873951  -0.21487665 -1.63588304 -3.13032257  0.89457041
  -0.32556438  1.81685833 -0.27783006  0.48865208 -1.46474963]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[[-4.66154099  2.23746363]
 [-2.76597405 -4.10399183]
 [-5.11898184  1.14089123]
 ...
 [ 1.09088909  4.316703  ]
 [-0.07718694 -1.40940865]
 [ 2.40251767  0.13651309]]
[[-6.75864380e+00  4.94215397e+00  2.13582502e-02 ... -2.50828669e+00
   2.44377352e+00 -2.60549756e+00]
 [-3.17348770e+00 -8.07678463e+00 -2.73136418e+00 ... -2.77830059e-01
   4.88652085e-01 -1.46474963e+00]
 [-1.72176072e+01 -7.99806914e+00  3.46817993e+00 ... -2.91272808e+00
   1.52315207e-01  2.93219325e-01]
 ...
 [-6.72912382e+00  1.17268948e+00  1.65670158e+01 ...  1.66252276e+00
  -3.10456199e-01  2.77744091e-01]

AnnData object with n_obs × n_vars = 12330 × 5286 
    obs: 'nCount_RNA', 'percent.mito', 'Cluster', 'Individual'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    obsm: 'X_umap', 'X_pca'
    layers: 'matrix', 'ambiguous', 'spliced', 'unspliced'

Filter and normalize the adata object, then calculate "moments" for each cell. You can define the number of genes kept with shared counts and n_top_genes. Use method "sklearn" rather than "umap" because "umap" throws an error. 

In [64]:
x=adata.layers['spliced']
print(x.sum(1).A1)

x=adata.layers['unspliced']
x.sum(1).A1

[5860 2107 4261 ... 3150 1943 1976]


array([3789, 1133, 1600, ..., 2298, 1341, 1939], dtype=uint64)

In [91]:
adata = adata[adata.obs['Condition'] == 'Control',:]

In [93]:
#scv.pp.filter_genes(adata,min_counts = 1)
adata


View of AnnData object with n_obs × n_vars = 7049 × 4844 
    obs: 'nCount_RNA', 'percent.mito', 'Cluster', 'Individual', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size', 'n_counts', 'velocity_self_transition', 'Condition'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'gene_count_corr', 'velocity_gamma', 'velocity_r2', 'velocity_genes'
    uns: 'Individual_colors', 'neighbors', 'velocity_params', 'velocity_graph', 'velocity_graph_neg', 'Cluster_colors'
    obsm: 'X_umap', 'X_pca', 'velocity_umap'
    layers: 'matrix', 'ambiguous', 'spliced', 'unspliced', 'Ms', 'Mu', 'velocity', 'variance_velocity'
    obsp: 'distances', 'connectivities'

In [94]:
scv.pp.normalize_per_cell(adata,enforce = True,use_initial_size=True) #enforce means recalculate
#adata.obs['n_counts']

Trying to set attribute `.obs` of view, copying.


Normalized count data: X, spliced, unspliced.


In [67]:
print(adata.layers['spliced'][1:100,1:5])
adata.n_vars

  (1, 0)	45.700775
  (1, 2)	0.617578
  (16, 0)	2.787606
  (19, 3)	0.91562283
  (20, 0)	1.4627571
  (26, 3)	1.8574202
  (29, 3)	1.3230267
  (30, 3)	0.5098818
  (34, 3)	8.514213
  (36, 3)	0.9933937
  (38, 3)	0.5767039
  (40, 0)	17.852781
  (42, 0)	1.6477771
  (44, 3)	0.99152225
  (46, 0)	0.4734617
  (53, 0)	0.46509367
  (53, 2)	11.162249
  (53, 3)	0.46509367
  (57, 3)	2.6925306
  (62, 0)	1.1226536
  (63, 3)	4.078791
  (65, 3)	0.3822077
  (71, 0)	29.495758
  (71, 1)	10.363375
  (71, 2)	12.754923
  (80, 3)	2.11876
  (82, 3)	0.8335445
  (83, 3)	2.144225
  (88, 3)	1.7840677
  (90, 2)	2.6058142
  (92, 3)	1.0380671
  (94, 3)	1.2453858


4844

In [470]:
scv.pp.filter_genes_dispersion(adata,flavor = 'seurat',n_top_genes=5000)
adata


Exctracted 5001 highly variable genes.


AnnData object with n_obs × n_vars = 6666 × 5001 
    obs: 'nCount_RNA', 'percent.mito', 'percent.ribo', 'Cluster', 'Individual', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size', 'n_counts'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'gene_count_corr', 'means', 'dispersions', 'dispersions_norm', 'highly_variable'
    uns: 'Cluster_colors', 'Individual_colors'
    obsm: 'X_umap', 'X_pca'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'

In [508]:
adata.write("adata.h5ad",compression = "gzip")